In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import jellyfish
import snowflake.connector
import time

import warnings
warnings.filterwarnings('ignore')


/home/rkothari/.local/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
main_start = time.time()

In [ ]:
data = pd.read_csv('~/Downloads/invoice_product_descriprions',sep=',')

In [ ]:
res = data[:50000]

In [ ]:

# cnx = snowflake.connector.connect(user='rkothari', password='Fall19@123', account='FF80747.east-us-2.azure', warehouse='DASCI_WH_XS', database='FDC_STAGE', schema='DASCI', role='DASCI_DEVELOPER')

# con = cnx.cursor()

# query = """SELECT DISTINCT pm.GTIN, i.INVOICE_PACKAGE_DESCRIPTION, pm.PRODUCT_DESCRIPTION, pm.INVOICE_UPC_CASE
#         FROM FDC_PROD_LIVE.FDC02.ENTITY_INVOICE i
#         JOIN PRODUCT_MASTER_CATALOG_MAPPED_TEMP pm on pm.INVOICE_ITEM_FTS_ID = i.INVOICE_ITEM_FTS_ID
#         WHERE INVOICE_PACKAGE_DESCRIPTION not in (SELECT INVOICE_PACKAGE_DESCRIPTION FROM INVOICE_NONPRODUCT_FILTER)
#             and i.INVOICE_DATE >= '2020-01-01'
#         GROUP BY pm.GTIN, i.INVOICE_PACKAGE_DESCRIPTION, pm.PRODUCT_DESCRIPTION, pm.INVOICE_UPC_CASE
#         ORDER BY pm.GTIN"""
# results = con.execute(query)
# res = pd.DataFrame.from_records(iter(results), columns=[y[0] for y in results.description])


    

# con.close()



In [ ]:
res.head()

In [ ]:

start = time.time()
res['fuzz_partial_ratio'] = res.apply(lambda x: fuzz.partial_ratio(x.INVOICE_PACKAGE_DESCRIPTION, x.PRODUCT_DESCRIPTION), axis=1)
end = time.time()
print('Time taken by fuzz_partial_ratio ', (end - start))

In [ ]:
res.head()

In [ ]:
res['INVOICE_SOUNDEX'] = res.apply(lambda x: jellyfish.soundex(x.INVOICE_PACKAGE_DESCRIPTION), axis=1) 
res['PRODUCT_DESCRIPTION_SOUNDEX'] = res.apply(lambda x: jellyfish.soundex(x.PRODUCT_DESCRIPTION), axis=1)

In [ ]:
start = time.time()
res['jellyfish_ratio'] = res.apply(lambda x: fuzz.partial_ratio(x.INVOICE_SOUNDEX, x.PRODUCT_DESCRIPTION_SOUNDEX), axis=1)
end = time.time()
print('Time taken by fuzz_partial_ratio ', (end - start))

In [ ]:
# Imported the inaugural corpus from nltk.corpus
import nltk
from nltk.corpus import inaugural
from nltk.tokenize import regexp_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

res['INVOICE_tokens'] = res.apply(lambda x: nltk.word_tokenize(x.INVOICE_PACKAGE_DESCRIPTION), axis = 1)

In [ ]:
res['product_tokens'] = res.apply(lambda x: nltk.word_tokenize(x.PRODUCT_DESCRIPTION), axis = 1)

In [ ]:
# set(nltk.ngrams(tokens1, n=3))

res['INVOICE_ngram'] = res['INVOICE_tokens'].apply(lambda r: list(nltk.ngrams(r,n=2)))
res['product_ngrams'] = res['product_tokens'].apply(lambda r: list(nltk.ngrams(r,n=2)))

In [ ]:
res['INVOICE_ngram'] = res['INVOICE_tokens'].apply(lambda r: set(nltk.ngrams(r,n=2)))
res['product_ngrams'] = res['product_tokens'].apply(lambda r: set(nltk.ngrams(r,n=2)))

In [ ]:
res['fuzz_ratio_ngrams'] = res.apply(lambda x: fuzz.partial_ratio(x.INVOICE_ngram, x.product_ngrams), axis=1)


In [ ]:
# nltk.jaccard_distance(ng1_tokens, ng3_tokens)
# res['jellyfish_ratio'] = res.apply(lambda x: fuzz.partial_ratio(x.INVOICE_SOUNDEX, x.PRODUCT_DESCRIPTION_SOUNDEX), axis=1)

res['jaccard_distance'] = res.apply(lambda x: 1-nltk.jaccard_distance(set(x.INVOICE_tokens) , set(x.product_tokens)),axis =1)

In [ ]:
res.head()

In [ ]:

res['string_invoice_ngram'] = [','.join(map(str, l)) for l in res['INVOICE_tokens']]
res['string_product_ngram'] = [','.join(map(str, l)) for l in res['product_tokens']]


In [ ]:
res.columns

In [ ]:
def similarity(word1, word2,soundex1, soundex2, ngram1, ngram2):
    
    score_1 = fuzz.partial_ratio(word1, word2) # score from fuzzywuzzy
    score_2 = fuzz.partial_ratio(soundex1,soundex2) #jellyfish score
    score_3 = fuzz.partial_ratio(ngram1, ngram2)  #fuzzy_wuzzy on ngram
    
        
    score = 0.14*score_1+ 0.64*score_2 +0.18*score_3# customize your own weights 
    
    return score

In [ ]:
start = time.time()
res['weight_match'] = res.apply(lambda x: similarity(x.INVOICE_PACKAGE_DESCRIPTION, x.PRODUCT_DESCRIPTION, 
                                                     x.INVOICE_SOUNDEX ,x.PRODUCT_DESCRIPTION_SOUNDEX, 
                                                     x.string_invoice_ngram, x.string_product_ngram), axis = 1)
end = time.time()
print('Time taken by fuzz_partial_ratio ', (end - start))

In [ ]:
res.columns

In [ ]:
master_wieghts = res[["INVOICE_PACKAGE_DESCRIPTION","PRODUCT_DESCRIPTION","fuzz_partial_ratio","jellyfish_ratio","fuzz_ratio_ngrams", "weight_match"]]

In [ ]:
master_wieghts.sort_values(by=['weight_match'],ascending=False,inplace = True)

In [ ]:
master_wieghts.head()

In [ ]:
temp = res[["fuzz_partial_ratio","jellyfish_ratio","fuzz_ratio_ngrams"]]

In [ ]:
import tensorflow as tf

In [ ]:
pip uninstall tensorflow

In [ ]:
# res.columns
master_wieghts.to_csv("~/Downloads/master_wieghts.csv")

In [ ]:
main_end = time.time()

In [ ]:
print("total time taken:" , (main_end - main_start ))

In [ ]:
print(np.dot([2.5,-5.0,-1.2,0.5,2.0,0.7] ,[3,2,1,3,0,4.19])+0.1)


In [ ]:
pd.options.display.max_colwidth = 1000

In [ ]:
res[res['INVOICE_PACKAGE_DESCRIPTION'] == 'OB DRY CIDER 4/6 CAN']

In [ ]:
res[res['INVOICE_PACKAGE_DESCRIPTION'] == 'OB DRY CIDER 4/6 CAN']

In [ ]:
w1 = set('Data is the new oil of the digital economy')
w2 = set('Data is a new oil')
 
print(nltk.jaccard_distance(w1, w2))
 

In [ ]:
def Jaccard_Similarity(doc1, doc2): 
    
    # List the unique words in a document
#     words_doc1 = set(doc1.lower().split()) 
#     words_doc2 = set(doc2.lower().split())
    
    # Find the intersection of words list of doc1 & doc2
    intersection = doc1.intersection(doc2)
    print(intersection)
    # Find the union of words list of doc1 & doc2
    union = doc1.union(doc2)
    print(union)
    # Calculate Jaccard similarity score 
    # using length of intersection set divided by length of union set
    return float(len(intersection)) / len(union)

In [6]:
jellyfish_main_str = jellyfish.soundex('11221250')
jellyfish_comp_str = jellyfish.soundex('1112321240')


In [7]:
jellyfish_main_str

'1000'

In [8]:
jellyfish_comp_str

'1000'

In [9]:
print(fuzz.partial_ratio(jellyfish_main_str, jellyfish_comp_str), end='--->')
print(fuzz.partial_ratio('1122121250','112287250'))

100--->67


In [ ]:
w1 = set(['BUD', '16OZ', '4/6', 'CAN'])
w2 = set(['BUDWEISER', 'BEER', '12', 'CT', '12', 'OZ', 'CAN', 'IN', 'BOX', '144', 'OZ'])

Jaccard_Similarity(w1,w2)
print(1-nltk.cjaccard_distance(w1, w2))

In [10]:
import re
from nltk.util import ngrams
 
s = "Natural-language processing (NLP) is an area of computer science " \
    "and artificial intelligence concerned with the interactions " \
    "between computers and human (natural) languages."
 
s = s.lower()
s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
tokens = [token for token in s.split(" ") if token != ""]
output = list(ngrams(tokens, 5))

In [11]:
# query = """SELECT DISTINCT pm.GTIN, i.INVOICE_PACKAGE_DESCRIPTION, pm.PRODUCT_DESCRIPTION, pm.GTIN
#         FROM FDC_PROD_LIVE.FDC02.ENTITY_INVOICE i
#         JOIN PRODUCT_MASTER_CATALOG_MAPPED_TEMP pm on pm.INVOICE_ITEM_FTS_ID = i.INVOICE_ITEM_FTS_ID
#         WHERE INVOICE_PACKAGE_DESCRIPTION not in (SELECT INVOICE_PACKAGE_DESCRIPTION FROM INVOICE_NONPRODUCT_FILTER)
#             and i.INVOICE_DATE >= '2020-01-01'
#         GROUP BY pm.GTIN, i.INVOICE_PACKAGE_DESCRIPTION, pm.PRODUCT_DESCRIPTION
#         ORDER BY pm.GTIN"""
 
# mapped01 = pd.read_sql(query, cnx, coerce_float = False)
 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import CountVectorizer
 
tokenizer = CountVectorizer(strip_accents='unicode',analyzer='char_wb',ngram_range=(3,3))
featurenames = []
sparse = []
for i in central.itertuples():
    x = tokenizer.fit_transform([i.PRODUCT_DESCRIPTION])
    sparse.append(x.toarray())
    featurenames.append(tokenizer.get_feature_names())
 
card_docs = [TaggedDocument(row, [l]) for l, row in enumerate(featurenames)]
model = Doc2Vec(vector_size=50, min_count=0, epochs=100, dm=1)
model.build_vocab(card_docs)
model.train(card_docs, total_examples=model.corpus_count, epochs=model.epochs)
 
ranks = []
for i in mapped01.itertuples():
    inferred_vector = model.infer_vector(i.INVOICE_PACKAGE_DESCRIPTION.split(' '))
    sims = model.wv.most_similar([inferred_vector], topn=len(card2vec))
    print('Tested Line: ', i.INVOICE_PACKAGE_DESCRIPTION)
    print(sims)

[('natural', 'language', 'processing', 'nlp', 'is'),
 ('language', 'processing', 'nlp', 'is', 'an'),
 ('processing', 'nlp', 'is', 'an', 'area'),
 ('nlp', 'is', 'an', 'area', 'of'),
 ('is', 'an', 'area', 'of', 'computer'),
 ('an', 'area', 'of', 'computer', 'science'),
 ('area', 'of', 'computer', 'science', 'and'),
 ('of', 'computer', 'science', 'and', 'artificial'),
 ('computer', 'science', 'and', 'artificial', 'intelligence'),
 ('science', 'and', 'artificial', 'intelligence', 'concerned'),
 ('and', 'artificial', 'intelligence', 'concerned', 'with'),
 ('artificial', 'intelligence', 'concerned', 'with', 'the'),
 ('intelligence', 'concerned', 'with', 'the', 'interactions'),
 ('concerned', 'with', 'the', 'interactions', 'between'),
 ('with', 'the', 'interactions', 'between', 'computers'),
 ('the', 'interactions', 'between', 'computers', 'and'),
 ('interactions', 'between', 'computers', 'and', 'human'),
 ('between', 'computers', 'and', 'human', 'natural'),
 ('computers', 'and', 'human', 'na